In [5]:
# Bringing in the data

import pandas as pd

# Read the CSV file into a DataFrame
df_mm = pd.read_csv("C:/Users/hagen/Downloads/NBA DATA/Teams/Transformed Data/mm_data.csv")

df_p = pd.read_csv("C:/Users/hagen/Downloads/NBA DATA/Teams/Transformed Data/p_data.csv")

In [23]:
import pandas as pd
import numpy as np
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split

def multi_SVM_top5(df, selected_features, C, method):
    """
    Custom cross-validation for predicting NBA champion using XGBoostRanker,
    with group information to rank the teams by their predicted level (score).
    """
    results = []

    # Iterate over the years you want to predict (2014 to 2023)
    for test_year in range(2014, 2024):
        
        # Training data: the previous 5 years of data (e.g., 2009-2013 for 2014 prediction)
        train_data = df[df['Year'].isin(range(test_year - 5, test_year))]
        
        # Test data: current year
        test_data = df[df['Year'] == test_year]
        
        # Prepare features and target for training and testing
        X_train = train_data[selected_features]
        y_train = train_data['Levels']
        
        X_test = test_data[selected_features]
        y_test = test_data['Levels']

        # Initialize the SVC model with probability=True
        model = SVC(probability=True, kernel='sigmoid', C=C, decision_function_shape=method, class_weight='balanced')

        # Train the model using SVM
        model.fit(X_train, y_train)
        
        # Make predictions on the test data
        y_pred = model.predict_proba(X_test)

        # Rank teams based on predicted scores
        ranked_teams = test_data[['Team']].copy()
        ranked_teams['Score'] = y_pred[:, 4]
        ranked_teams = ranked_teams.sort_values(by='Score', ascending=False)
        
        # Get the actual champion team (level 4) from the test data
        actual_champion = test_data[test_data['Levels'] == 4]['Team'].values[0]

        # Get top 5 teams and their scores
        top_teams = ranked_teams.head(5)
        top_teams_list = top_teams.apply(lambda row: f"{row['Team']}: {row['Score']:.4f}", axis=1).tolist()
        
        # Store results
        results.append([test_year, actual_champion] + top_teams_list)
    
    # Create dataframe with results
    columns = ['Year', 'Actual Champion', 'First Pick', 'Second Pick', 'Third Pick', 'Fourth Pick', 'Fifth Pick']
    results_df = pd.DataFrame(results, columns=columns)
    results_df = results_df.sort_values(by="Year", ascending=False)

    # Compute accuracy
    accuracy = (results_df['Actual Champion'] == results_df['First Pick'].str.split(': ').str[0]).mean()
    print(f"Model Accuracy: {accuracy * 100:.2f}%")
    
    return results_df

In [25]:
selected_features = ['W', 'PW', 'SRS', 'NRtg', 'MOV', 'eFG%', 'FG%', 'TS%']

multi_SVM_top5(df_mm, selected_features, C=1, method='ovo')

Model Accuracy: 50.00%


,Year,Actual Champion,First Pick,Second Pick,Third Pick,Fourth Pick,Fifth Pick
9,2023,Boston Celtics,Boston Celtics: 0.0451,Oklahoma City Thunder: 0.0442,Indiana Pacers: 0.0426,Denver Nuggets: 0.0421,Minnesota Timberwolves: 0.0421
8,2022,Denver Nuggets,Denver Nuggets: 0.0529,Boston Celtics: 0.0525,Philadelphia 76ers: 0.0523,Cleveland Cavaliers: 0.0504,Sacramento Kings: 0.0502
7,2021,Golden State Warriors,Phoenix Suns: 0.0565,Utah Jazz: 0.0549,Golden State Warriors: 0.0545,Boston Celtics: 0.0544,Miami Heat: 0.0535
6,2020,Milwaukee Bucks,Brooklyn Nets: 0.0468,Phoenix Suns: 0.0466,Utah Jazz: 0.0464,Los Angeles Clippers: 0.0462,Milwaukee Bucks: 0.0462
5,2019,Los Angeles Lakers,Milwaukee Bucks: 0.0459,Los Angeles Lakers: 0.0447,Los Angeles Clippers: 0.0434,Utah Jazz: 0.0434,Miami Heat: 0.0433
4,2018,Toronto Raptors,Golden State Warriors: 0.0391,Milwaukee Bucks: 0.0385,Toronto Raptors: 0.0376,Utah Jazz: 0.0363,Houston Rockets: 0.0356
3,2017,Golden State Warriors,Golden State Warriors: 0.0447,Houston Rockets: 0.0438,Toronto Raptors: 0.0433,Philadelphia 76ers: 0.0419,Cleveland Cavaliers: 0.0416
2,2016,Golden State Warriors,Golden State Warriors: 0.0388,Houston Rockets: 0.0352,San Antonio Spurs: 0.0350,Los Angeles Clippers: 0.0346,Cleveland Cavaliers: 0.0342
1,2015,Cleveland Cavaliers,Golden State Warriors: 0.0501,San Antonio Spurs: 0.0490,Oklahoma City Thunder: 0.0464,Cleveland Cavaliers: 0.0438,Los Angeles Clippers: 0.0404
0,2014,Golden State Warriors,Golden State Warriors: 0.0369,Los Angeles Clippers: 0.0362,Atlanta Hawks: 0.0357,San Antonio Spurs: 0.0355,Cleveland Cavaliers: 0.0347


In [27]:
selected_features = ['W', 'PW', 'SRS', 'NRtg', 'MOV', 'eFG%', 'FG%', 'TS%']

multi_SVM_top5(df_mm, selected_features, C=1, method='ovr')

Model Accuracy: 50.00%


,Year,Actual Champion,First Pick,Second Pick,Third Pick,Fourth Pick,Fifth Pick
9,2023,Boston Celtics,Boston Celtics: 0.0454,Oklahoma City Thunder: 0.0445,Indiana Pacers: 0.0431,Denver Nuggets: 0.0426,Minnesota Timberwolves: 0.0425
8,2022,Denver Nuggets,Denver Nuggets: 0.0438,Philadelphia 76ers: 0.0432,Boston Celtics: 0.0431,Sacramento Kings: 0.0428,Cleveland Cavaliers: 0.0420
7,2021,Golden State Warriors,Phoenix Suns: 0.0464,Utah Jazz: 0.0455,Golden State Warriors: 0.0451,Denver Nuggets: 0.0450,Boston Celtics: 0.0447
6,2020,Milwaukee Bucks,Brooklyn Nets: 0.0479,Phoenix Suns: 0.0476,Utah Jazz: 0.0472,Los Angeles Clippers: 0.0471,Milwaukee Bucks: 0.0470
5,2019,Los Angeles Lakers,Milwaukee Bucks: 0.0401,Los Angeles Lakers: 0.0397,Los Angeles Clippers: 0.0393,Utah Jazz: 0.0392,Miami Heat: 0.0392
4,2018,Toronto Raptors,Golden State Warriors: 0.0461,Milwaukee Bucks: 0.0456,Toronto Raptors: 0.0449,Utah Jazz: 0.0437,Houston Rockets: 0.0430
3,2017,Golden State Warriors,Golden State Warriors: 0.0522,Houston Rockets: 0.0518,Toronto Raptors: 0.0512,Philadelphia 76ers: 0.0471,Utah Jazz: 0.0430
2,2016,Golden State Warriors,Golden State Warriors: 0.0414,Houston Rockets: 0.0382,San Antonio Spurs: 0.0380,Los Angeles Clippers: 0.0376,Cleveland Cavaliers: 0.0372
1,2015,Cleveland Cavaliers,Golden State Warriors: 0.0353,San Antonio Spurs: 0.0344,Oklahoma City Thunder: 0.0334,Cleveland Cavaliers: 0.0322,Los Angeles Clippers: 0.0317
0,2014,Golden State Warriors,Golden State Warriors: 0.0371,Los Angeles Clippers: 0.0363,Atlanta Hawks: 0.0358,San Antonio Spurs: 0.0356,Cleveland Cavaliers: 0.0348


In [29]:
selected_features = ['W', 'PW', 'SRS', 'NRtg', 'MOV', 'eFG%', 'FG%', 'TS%']

multi_SVM_top5(df_p, selected_features, C=1, method='ovo')

Model Accuracy: 40.00%


,Year,Actual Champion,First Pick,Second Pick,Third Pick,Fourth Pick,Fifth Pick
9,2023,Boston Celtics,Boston Celtics: 0.0414,Oklahoma City Thunder: 0.0413,Denver Nuggets: 0.0398,Minnesota Timberwolves: 0.0394,Los Angeles Clippers: 0.0386
8,2022,Denver Nuggets,Philadelphia 76ers: 0.0458,Denver Nuggets: 0.0456,Boston Celtics: 0.0454,Cleveland Cavaliers: 0.0452,Sacramento Kings: 0.0448
7,2021,Golden State Warriors,Phoenix Suns: 0.0574,Utah Jazz: 0.0561,Golden State Warriors: 0.0555,Miami Heat: 0.0540,Boston Celtics: 0.0534
6,2020,Milwaukee Bucks,Phoenix Suns: 0.0364,Los Angeles Clippers: 0.0362,Brooklyn Nets: 0.0359,Milwaukee Bucks: 0.0359,Utah Jazz: 0.0356
5,2019,Los Angeles Lakers,Milwaukee Bucks: 0.0416,Los Angeles Lakers: 0.0403,Los Angeles Clippers: 0.0396,Miami Heat: 0.0391,Utah Jazz: 0.0390
4,2018,Toronto Raptors,Milwaukee Bucks: 0.0536,Golden State Warriors: 0.0536,Toronto Raptors: 0.0528,Utah Jazz: 0.0505,Houston Rockets: 0.0483
3,2017,Golden State Warriors,Golden State Warriors: 0.0451,Toronto Raptors: 0.0443,Houston Rockets: 0.0442,Philadelphia 76ers: 0.0427,Utah Jazz: 0.0406
2,2016,Golden State Warriors,Golden State Warriors: 0.0554,Los Angeles Clippers: 0.0537,Houston Rockets: 0.0536,San Antonio Spurs: 0.0531,Cleveland Cavaliers: 0.0528
1,2015,Cleveland Cavaliers,Golden State Warriors: 0.0440,San Antonio Spurs: 0.0437,Oklahoma City Thunder: 0.0432,Cleveland Cavaliers: 0.0425,Los Angeles Clippers: 0.0421
0,2014,Golden State Warriors,Golden State Warriors: 0.0463,Los Angeles Clippers: 0.0459,Atlanta Hawks: 0.0454,San Antonio Spurs: 0.0452,Cleveland Cavaliers: 0.0442


In [31]:
selected_features = ['W', 'PW', 'SRS', 'NRtg', 'MOV', 'eFG%', 'FG%', 'TS%']

multi_SVM_top5(df_p, selected_features, C=1, method='ovr')

Model Accuracy: 40.00%


,Year,Actual Champion,First Pick,Second Pick,Third Pick,Fourth Pick,Fifth Pick
9,2023,Boston Celtics,Boston Celtics: 0.0391,Oklahoma City Thunder: 0.0390,Denver Nuggets: 0.0378,Minnesota Timberwolves: 0.0375,Los Angeles Clippers: 0.0368
8,2022,Denver Nuggets,Philadelphia 76ers: 0.0436,Denver Nuggets: 0.0434,Boston Celtics: 0.0433,Cleveland Cavaliers: 0.0430,Sacramento Kings: 0.0425
7,2021,Golden State Warriors,Phoenix Suns: 0.0539,Utah Jazz: 0.0514,Golden State Warriors: 0.0502,Boston Celtics: 0.0470,Miami Heat: 0.0465
6,2020,Milwaukee Bucks,Phoenix Suns: 0.0391,Los Angeles Clippers: 0.0389,Brooklyn Nets: 0.0386,Milwaukee Bucks: 0.0386,Utah Jazz: 0.0383
5,2019,Los Angeles Lakers,Milwaukee Bucks: 0.0434,Los Angeles Lakers: 0.0420,Los Angeles Clippers: 0.0412,Miami Heat: 0.0406,Utah Jazz: 0.0405
4,2018,Toronto Raptors,Milwaukee Bucks: 0.0586,Golden State Warriors: 0.0585,Toronto Raptors: 0.0573,Utah Jazz: 0.0533,Houston Rockets: 0.0496
3,2017,Golden State Warriors,Golden State Warriors: 0.0417,Toronto Raptors: 0.0411,Houston Rockets: 0.0410,Philadelphia 76ers: 0.0399,Utah Jazz: 0.0382
2,2016,Golden State Warriors,Golden State Warriors: 0.0418,Los Angeles Clippers: 0.0401,Houston Rockets: 0.0401,San Antonio Spurs: 0.0398,Cleveland Cavaliers: 0.0394
1,2015,Cleveland Cavaliers,Golden State Warriors: 0.0437,San Antonio Spurs: 0.0434,Oklahoma City Thunder: 0.0429,Cleveland Cavaliers: 0.0422,Los Angeles Clippers: 0.0418
0,2014,Golden State Warriors,Golden State Warriors: 0.0417,Los Angeles Clippers: 0.0412,Atlanta Hawks: 0.0407,San Antonio Spurs: 0.0406,Cleveland Cavaliers: 0.0395
